## Kubeflow pipelines

This notebook goes through the steps of using Kubeflow pipelines using the Python3 interpreter (command-line).

### 1. Create cluster

%%bash
gcloud config set compute/zone us-central1-b
gcloud container clusters create lakpipeline \
  --zone us-central1-b \
  --scopes cloud-platform \
  --enable-cloud-logging \
  --enable-cloud-monitoring \
  --machine-type n1-standard-2 \
  --num-nodes 4
kubectl create clusterrolebinding ml-pipeline-admin-binding --clusterrole=cluster-admin --user=$(gcloud config get-value account)

Go the [Google Kubernetes Engine section of the GCP console](https://console.cloud.google.com/kubernetes) and make sure that the cluster is started and ready.  This will take about 3 minutes.

### 2. Deploy Kubeflow pipeline to cluster

In [ ]:
%%bash
PIPELINE_VERSION=0.1.2
kubectl create -f https://storage.googleapis.com/ml-pipeline/release/$PIPELINE_VERSION/bootstrapper.yaml

### 3. Install local interpreter

In [ ]:
%%bash
PIPELINE_VERSION=0.1.2
pip3 install python-dateutil https://storage.googleapis.com/ml-pipeline/release/$PIPELINE_VERSION/kfp.tar.gz --upgrade

After pip install, always <b>Reset Session</b> so that the new package gets picked up.

### 4. Set up port forward

In [ ]:
%%bash
export NAMESPACE=kubeflow
kubectl port-forward -n ${NAMESPACE} $(kubectl get pods -n ${NAMESPACE} --selector=service=ambassador -o jsonpath='{.items[0].metadata.name}') 8085:80

Now visit https://localhost:8085/pipeline

### 5. Do the DSL compile

In [21]:
%%bash
OUTDIR=dsl
rm -rf $OUTDIR
mkdir -p $OUTDIR
dsl-compile --py mlp_babyweight.py --output $OUTDIR/mlp_babyweight.tar.gz
ls -l $OUTDIR

total 4
-rw-r--r-- 1 root root 1015 Nov 12 09:52 mlp_babyweight.tar.gz


### 5. Upload and execute pipeline

Start by navigating to localhost:8085 (as in port forward)